# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from cassandra.cluster import Cluster
from nosql_queries import *
from conf import *

#### Creating list of filepaths to process original event csv data files

In [2]:
file_path_list = []

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Creates a list of filepath where the files are located 
for root, dirs, files in os.walk(filepath):
    files = glob.glob(os.path.join(root,'*.csv'))
    for f in files:
        if '.ipynb_checkpoints' in f:
            pass
        else:
            file_path_list.append(os.path.abspath(f))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
#             print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(str(len(full_data_rows_list)) + ' - total number of rows')
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        else:
            writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8056 - total number of rows


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(str(sum(1 for line in f)) + ' - number of rows in csv file')

6821 - number of rows in csv file


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
try:
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
try:
    session.execute(
    """
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

#### Set file (consolidated csv) variable

In [8]:
file = 'event_datafile_new.csv'

### Task 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### Used the columns, session_id and item_in_session,as the primary keys as these are the columns in the WHERE clause.
#### Then added the columns: artist,  song, and length, to give the details being requested.

In [9]:
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS song_details_library (session_id int, item_in_session int, 
    artist text, song text, length double, PRIMARY KEY ((session_id, item_in_session)))
    """)
except Exception as e:
    print(e)         

#### Inserted data in this table based on the order of columns in the 'create table' statement

In [10]:
# Reads the consolidated CSV file and extracts the data for ingestion
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_details_library (session_id, item_in_session, artist, song, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)

In [11]:
query = """
    SELECT artist, song, length FROM song_details_library
    WHERE session_id = 338 AND item_in_session = 4
    """

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, ',', row.song, ',', row.length)

Faithless , Music Matters (Mark Knight Dub) , 495.3073


### Task 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Used the columns, user_id and session_id,as the primary keys as these are the columns in the WHERE clause, followed by the clustering column, item_in_session. 
#### Then added the columns: artist,  song, and user, to give the details being requested.

In [12]:
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS session_details_library (user_id int, session_id int, 
    item_in_session int, artist text, song text, first_name text, last_name text,
    PRIMARY KEY ((user_id, session_id), item_in_session))
    """)
except Exception as e:
    print(e)             

In [13]:
# Reads the consolidated CSV file and extracts the data for ingestion
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_details_library (user_id, session_id, item_in_session, artist, song, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as e:
            print(e)

In [14]:
query = """
    SELECT artist, song, first_name, last_name FROM session_details_library
    WHERE user_id = 10 AND session_id = 182
    """

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, ',', row.song, ',', row.first_name + ' ' + row.last_name)

Down To The Bone , Keep On Keepin' On , Sylvie Cruz
Three Drives , Greece 2000 , Sylvie Cruz
Sebastien Tellier , Kilometer , Sylvie Cruz
Lonnie Gordon , Catch You Baby (Steve Pitron & Max Sanna Radio Edit) , Sylvie Cruz


### Task 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Used the column, song, as the primary key as this is the column in the WHERE clause, followed by the clustering column, user_id, to give make each entry unique to the song and user.

#### Then added the column user, to complete the details being requested.

In [15]:
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS user_history_library (song text, user_id int, first_name text, last_name text, 
    PRIMARY KEY ((song), user_id))
    """)
except Exception as e:
    print(e)   

In [16]:
# Reads the consolidated CSV file and extracts the data for ingestion
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_history_library (song, user_id, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s)"
        try:
            session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
            print(e)

In [17]:
query = """
    SELECT first_name, last_name FROM user_history_library
    WHERE song = 'All Hands Against His Own'
    """

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.first_name + ' ' + row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [18]:
queries = ["DROP TABLE IF EXISTS song_details_library", "DROP TABLE IF EXISTS session_details_library",
          "DROP TABLE IF EXISTS user_history_library"]
try:
    for query in queries:
        rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()